In [70]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.transform import rowcol, xy
from shapely.geometry import Point
from pyproj import Geod

In [2]:
raster_path = r'C:\Users\somadder\Thesis\SM\SM.tif'
peaks_path = r'C:\Users\somadder\Thesis\SM\sm_p3_peaks.shp'
gnis_path = r'C:\Users\somadder\Thesis\SM\GNIS_Summits.shp'

In [5]:
peaks = gpd.read_file(peaks_path)

In [7]:
gnis_s = gpd.read_file(gnis_path)

In [76]:
def calculate_distance(x1, y1, x2, y2):
    # Simple Euclidean distance calculation (valid for UTM coordinates)
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [66]:
def move_upslope(raster, start_row, start_col, window_size=3):
    height, width = raster.shape
    current_row, current_col = start_row, start_col
    current_value = raster[current_row, current_col]
    while True:
        window_rows = slice(max(0, current_row - 1), min(height, current_row + 2))
        window_cols = slice(max(0, current_col - 1), min(width, current_col + 2))
        window = raster[window_rows, window_cols]
        # print(window_rows, window_cols)
        # print(window)
        max_value = np.max(window)
        
        # pad_width = window_size//2
        # center_cell_mask = np.ones((window_size, window_size), dtype=bool)
        # center_cell_mask[pad_width, pad_width] = False
        # second_highest = max(window[center_cell_mask].ravel())

        # print('max: ', max_value)
        # print('second: ', second_highest)

        if max_value <= current_value:
            return current_row, current_col, current_value

        max_position = np.unravel_index(np.argmax(window), window.shape)
        # print(max_position)
        current_row = max(0, min(height - 1, current_row - 1 + max_position[0]))
        current_col = max(0, min(width - 1, current_col - 1 + max_position[1]))
        current_value = raster[current_row, current_col]

In [77]:
with rasterio.open(raster_path) as src:
    raster = src.read(1)
    transform = src.transform
    crs = src.crs

    gnis_features = gnis_s.to_crs(crs)

    new_geometries = []
    new_elevations = []
    new_ids = []
    distances = []

    for idx, feature in gnis_features.iterrows():
        x, y = feature.geometry.x, feature.geometry.y
        row, col = rowcol(transform, x, y)
        # print(row, col)
        # print(raster[4645, 6611])
        # print(feature['FEATURE_NA'])
        new_row, new_col, new_elv = move_upslope(raster, row, col)
        # print(new_row, new_col)
        new_x, new_y = xy(transform, new_row, new_col)
        # print(new_x, new_y)
        # print(Point(transform * (new_row, new_col)))
        new_geometries.append(Point(new_x, new_y))
        new_elevations.append(new_elv)
        new_ids.append(str(new_row)+str(new_col))
        distances.append(calculate_distance(x, y, new_x, new_y))
        
    new_gnis_features = gpd.GeoDataFrame(gnis_features.drop(columns='geometry'), geometry=new_geometries, crs=crs)
    new_gnis_features['n_ids'] = new_ids
    new_gnis_features['distance_moved'] = distances
    new_gnis_features.to_crs(gnis_s.crs)
    new_gnis_features.to_file(r'C:\Users\somadder\Thesis\SM\new_gnis_features.shp')

C:\Users\somadder\AppData\Local\Temp\ipykernel_10436\2741870783.py:33: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  new_gnis_features.to_file(r'C:\Users\somadder\Thesis\SM\new_gnis_features.shp')
